## Part 2 La$_2$Ti$_2$O$_7$ surface

Supervisor: Prof. Aschauer

Assistants: Dr. Xing Wang

Room:	N216

### 1. Stoichiometric La$_2$Ti$_2$O$_7$ surface

ASE provides a `surface` function to setup the surface. Here we focus on the (001) surface of La$_2$Ti$_2$O$_7$. In the atomic structure, we kept the bottom two atomic layers fixed at their bulk positions, which is an (admittedly pretty drastic) approximation of the surface of a very large solid.


We should build a symmetric and stoichiometric surface slab if it is possible. Please discuss with me when you doing this.

Here (001) surface:



In [14]:
from ase.io import read
from ase.build import surface, add_adsorbate
from ase.constraints import FixAtoms
from ase.geometry import get_layers
from ase.visualize import view
from xespresso import Espresso
from xespresso.tools import mypool, fix_layers, dipole_correction
from x3dase.visualize import view_x3d_n
# read the optmized La2Ti2O7 structure
lto = read('datas/La2Ti2O7.cif')
# lto = Espresso(label = 'bulk/lto').results['atoms']
slab = surface(lto, (0, 0, 1), 1, vacuum = 5.0)
slab.pbc = [True, True, True]
# view(slab)
layers = get_layers(slab, (0, 0, 1), 1.0)[0]
index = [j for j in range(len(slab)) if layers[j] in range(2, 11)]
slab = slab[index]
# 
view_x3d_n(slab, bond = 0.8, label = True, polyhedra = {'Ti': ['O']}, output = 'htmls/lto-surface.html')

Then build the Espresso calculator.

In [ ]:
# For this big bulk system, we use the "empi" partition
queue = {'nodes': 4, 'ntasks-per-node': 20, 'partition': 'empi', 'mem-per-cpu': '5G', 'time': '23:59:00'}

input_ntyp = {
    'Hubbard_U': {'Ti': 3.0},
    'starting_magnetization': {'Ti': 0.0}}

calc = Espresso(label = 'surface/lto-010',
                pseudopotentials = mypsudo,
                calculation = 'relax',   # allow atoms to move
                ecutwfc = 40,
                occupations = 'smearing',
                degauss = 0.01,
                kpts = (2, 4, 1),        # k-points, 
                electron_maxstep = 200,
                mixing_mode = 'local-TF',
                nspin = 2,
                lda_plus_u = True,
                input_data = {'input_ntyp': input_ntyp},
                queue = queue,
                parallel = '-nk 5',)
slab.calc = calc
atoms.get_potential_energy()
e = slab.calc.results['energy']
print('Energy {0:3.3f}'.format(e))

### 2. Defective surface

One of the most common defects on the surface iss vacancy. Here we only consider the oxygen vacancy site. There several oxygen atoms on the surface. Please remove each of the oxygen atoms. 

In [ ]:
slab_ov = slab.calc.results['atoms'].copy
del slab.info['species']

sites = [75, 83]
jobs = {}
for site in sites:
    slab_ov = slab.copy()
    del slab_ov[site]
    jobs['ov-%s'%site] = slab_ov
view(jobs.values())

The build the Espresso calculator and run the calculations. Which site is the most stable one? 